In [3]:
import pandas as pd 
import numpy as np 
import sklearn

In [5]:
# Creating dataframe 
df = pd.read_excel(r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\Workout-Analysis-API\Outputs\workout_data.xls',sheet_name = 'All_Workouts_Table')
df = df[['activity_id','sport_type','distance','workout_time_min','calories','total_elevation_gain','average_speed','max_speed','max_speed','average_heartrate','max_heartrate','avg_time_per_lap','lap_count']]
df

,activity_id,sport_type,distance,workout_time_min,calories,total_elevation_gain,average_speed,max_speed,max_speed,average_heartrate,max_heartrate,avg_time_per_lap,lap_count
0,8885873542,Run,5.50,41.83,449.0,48.0,2.429,4.048,4.048,151.4,164.0,6.966667,6
1,8879705626,Functional-Cardio Workout,0.00,31.97,344.0,0.0,0.000,0.000,0.000,146.7,178.0,5.318333,6
2,8875392009,Functional-Cardio Workout,0.00,16.20,194.0,0.0,0.000,0.000,0.000,150.0,179.0,5.393333,3
3,8869251524,Functional-Cardio Workout,0.00,20.45,246.0,0.0,0.000,0.000,0.000,147.2,168.0,4.076000,5
4,8862861001,Yoga,0.00,19.78,106.0,0.0,0.000,0.000,0.000,100.6,140.0,19.780000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,3517497833,Run,3.57,28.92,352.5,4.2,2.061,4.900,4.900,NaN,NaN,28.920000,1
663,3516568449,Ride,10.64,38.50,0.0,13.9,4.695,10.800,10.800,NaN,NaN,38.500000,1
664,3509004014,Run,4.43,36.25,443.1,15.6,2.065,4.700,4.700,NaN,NaN,36.270000,1
665,3496090180,Run,3.45,32.63,340.5,3.9,1.832,4.800,4.800,NaN,NaN,32.630000,1
